# ELMo Model with FAISS Evaluation

In [1]:
import pandas as pd
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import tensorflow
import sklearn
from tqdm import tqdm
import os
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Loading model for evaluation

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
accepted_answers_df = pd.read_csv('Accepted_Answers.csv')
accepted_answers_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
9995,"Spring Security Core authAjax, How do I ignore...",<p>When you make an un-authorized attempt to a...,5628199
9996,Dictionary+Queue Data Structure with Active Re...,<p>I'd start by just modeling the behavior you...,5888734
9997,Extend an existing list in Django settings,<p>You should be able to import form <code>dja...,5916209
9998,Python: How to deal with non-subscriptable obj...,<p>The way to fix it is to use dictionaries pr...,5977165


In [6]:
scored_answer_df = pd.read_csv('Scored_Answers.csv')
scored_answer_df

,Question,Relevant_Answer,question_id
0,in android how to send and receive special sms...,<p>For this you have to create your custom bro...,29165268
1,Python Salary Program Using Conditionals,<p>You're using full quotes where you should b...,29089187
2,How can I show a spinner in a ViewPager when t...,<p>I would do a check for this inside the <cod...,29169909
3,Android: Horizontal list view on custom adaptor,<blockquote>\n <p>layout whose one item cover...,28878917
4,"Even If Testcase failed It is going in ""onTest...",<p>Function <code>verify</code> won't make a t...,29300800
...,...,...,...
9995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
9996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
9997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
9998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


In [7]:
ground_truth_df = pd.concat([accepted_answers_df, scored_answer_df], axis=0, ignore_index=True)
ground_truth_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


In [8]:
all_answers_df = pd.read_csv('All_answers.csv')
all_answers_df

,question_id,question_title,answer_id,answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...
...,...,...,...,...
33424,73829515,Big Query first non null value record type,73830169,<p>Consider below option</p>\n<pre><code>selec...
33425,73829809,Change Wordpress colors based on logo main color,73829900,"<p>First of all, you need colors from the imag..."
33426,73831805,Why are methods not valid constructors?,73832791,<p>As pointed out in the comments on the origi...
33427,73834279,No ability to publish newer build for external...,73834304,<p>I believe that there is possible to do more...


In [9]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r' ', text)

def remove_punctuation_and_lower(text):
    text = text.lower()
    return text.translate(str.maketrans(' ', ' ', string.punctuation))

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    return [word for word in word_tokens if word not in stop_words]

def remove_usernames(text: str) -> str:
    """
    Remove usernames starting with '@' from the given text.

    Args:
        text (str): The input text containing usernames.

    Returns:
        str: The text with usernames removed.
    """
    cleaned_text = re.sub(r"@[A-Za-z0-9_]+", " ", text)
    return cleaned_text

def clean_text(text):
    text = remove_html_tags(text)
    text = remove_usernames(text)
    text = remove_urls(text)
    text = remove_punctuation_and_lower(text)
    text_without_newlines = text.replace('\n', ' ')
    return text_without_newlines


In [10]:
ground_truth_df['Clean_Relevant_Answer'] = ground_truth_df['Relevant_Answer'].apply(clean_text)
ground_truth_df

,Question,Relevant_Answer,question_id,Clean_Relevant_Answer
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519,you dont appear to be returning anything from ...
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802,perl to the rescue perl e print packb argv0 0...
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188,your variables are mutated from inside of an a...
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556,attributestokenidlastclaimdate is a uint so at...
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861,conn is not defined here either change connect...
...,...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549,i am using the version 0616 you can set the ...
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033,your question doesnt really have anything to d...
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186,using a break instruction is a possibility but...
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275,you can try using f800 for englishus locale as...


In [11]:
all_answers_df['clean_answer_body'] = all_answers_df['answer_body'].apply(clean_text)
all_answers_df.head()

,question_id,question_title,answer_id,answer_body,clean_answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...,thats excellent for something written from ...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...,if you are intending on reading a large number...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...,if you are querying a sql server database vers...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...,very roughly and from memory since i dont have...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...,thats definitely a good way to do it but you ...


In [12]:
# Group the rows by question_id and aggregate the other columns into lists
grouped_answers_df = all_answers_df.groupby('question_id').agg({
    'question_title': lambda x: list(x)[0],
    'answer_id': lambda x: list(x),
    'answer_body': lambda x: list(x),
    'clean_answer_body': lambda x: list(x)
})

In [13]:
# Reset the index and rename the columns
grouped_answers_df = grouped_answers_df.reset_index().rename(columns={
    'answer_id': 'answer_ids',
    'answer_body': 'answer_bodies',
    'clean_answer_body': 'clean_answer_bodies'
})
grouped_answers_df

,question_id,question_title,answer_ids,answer_bodies,clean_answer_bodies
0,930,How do I connect to a database and loop over a...,"[951, 65774, 20049, 940, 1022, 2369, 56179247,...",[<p>@Goyuix -- that's excellent for something ...,[ thats excellent for something written from...
1,22979,Detecting if SQL server is running,"[451160, 23019, 23011, 23010, 22991, 4217054]",[<p><code>System.Data.Sql.SqlDataSourceEnumera...,[systemdatasqlsqldatasourceenumerator will ret...
2,23027,CruiseControl.Net Build Publisher - Only publi...,"[32667, 32681, 23060, 7028931]",[<p>The default build publisher in CC.NET does...,[the default build publisher in ccnet does not...
3,34896,When is it best to sanitize user input?,"[34906, 34905, 34984, 34902, 36088, 34923, 349...",[<p>Users are evil!</p>\n\n<p>Well perhaps not...,[users are evil well perhaps not always but my...
4,37263,"Where does ""Change Management"" end and ""Projec...","[43187, 37565, 37277, 37291, 37292]","[<p><strong>What is change management, and how...",[what is change management and how does it app...
...,...,...,...,...,...
19995,73829515,Big Query first non null value record type,"[73829947, 73830169]",[<p>You may consider below scalar subquery for...,[you may consider below scalar subquery for yo...
19996,73829809,Change Wordpress colors based on logo main color,[73829900],"[<p>First of all, you need colors from the ima...",[first of all you need colors from the image y...
19997,73831805,Why are methods not valid constructors?,[73832791],[<p>As pointed out in the comments on the orig...,[as pointed out in the comments on the origina...
19998,73834279,No ability to publish newer build for external...,[73834304],[<p>I believe that there is possible to do mor...,[i believe that there is possible to do more e...


In [14]:
final_df = pd.merge(grouped_answers_df, ground_truth_df, on='question_id')
final_df.head()

,question_id,question_title,answer_ids,answer_bodies,clean_answer_bodies,Question,Relevant_Answer,Clean_Relevant_Answer
0,89226,Is there a tool to convert a .vim colour defin...,[89300],[<p>Here is a Visual Studio theme generator wh...,[here is a visual studio theme generator which...,Is there a tool to convert a .vim colour defin...,<p>Here is a Visual Studio theme generator whi...,here is a visual studio theme generator which ...
1,119792,"Subversion Berkeley DB broken, recovery failed","[119798, 119957, 119862, 2851717]",[<blockquote>\n <p>I've got a Subversion repo...,[ ive got a subversion repository backed by th...,"Subversion Berkeley DB broken, recovery failed",<blockquote>\n <p>I've got a Subversion repos...,ive got a subversion repository backed by the...
2,156089,Silverlight 2.0 DataGrid How to remove mouseov...,"[183936, 1681462]",[<p>Yeah You have to change the Style and Cont...,[yeah you have to change the style and control...,Silverlight 2.0 DataGrid How to remove mouseov...,<p>The short answer is to use styles. The lon...,the short answer is to use styles the long an...
3,181241,"Why does the ""at"" command always warn me that ...","[182525, 181245, 181252, 55071622]",[<p>Does the warning have any harmful effect a...,[does the warning have any harmful effect asid...,"Why does the ""at"" command always warn me that ...",<p>It serves as a good warning to those of us ...,it serves as a good warning to those of us tha...
4,236359,What is your convention to distinguish between...,"[237976, 236596, 236402]",[<p>I'd like to make the suggestion that when ...,[id like to make the suggestion that when you ...,What is your convention to distinguish between...,<pre><code>use no underscores for the external...,use no underscores for the external api one un...


In [15]:
## Using unprocessed data to create question and answer pairs

from sklearn.model_selection import train_test_split

# Create question-answer pairs and labels
input_pairs = []
labels = []

questions_list = final_df['question_title'][:2000].values
answers_list = final_df['clean_answer_bodies'][:2000].values
correct_answers_list = final_df['Clean_Relevant_Answer'][:2000].values

for question, ans_list in zip(questions_list, answers_list):
    for answer in ans_list:
        input_pairs.append((question, answer))
        labels.append(1 if answer in correct_answers_list else 0)


In [16]:
# Split data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(input_pairs, labels, test_size=0.2, random_state=42, stratify=labels)

In [17]:
train_x = np.array([' '.join(t) for t in train_data]).reshape(len(train_data),1)
val_x = np.array([' '.join(t) for t in val_data]).reshape(len(val_data),1)

train_y = np.array(train_labels).reshape(len(train_labels),1)
val_y = np.array(val_labels).reshape(len(val_labels),1)

In [18]:
del input_pairs
del labels
del train_data
del val_data

In [19]:
tf.compat.v1.disable_eager_execution()

In [20]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, Layer
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [21]:
class ElmoEmbeddingLayer(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        self.dimensions=1024
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=self.trainable, name=f'{self.name}_module')
        if self.trainable:
            self._trainable_weights.extend(tf.compat.v1.trainable_variables(scope=f'^{self.name}_module/.*'))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, dtype=tf.string), axis=1), as_dict=True, signature='default')['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [22]:
def create_model(train_elmo=False):
  ## input layer and embedding for questions
  input_text = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
  embedding = ElmoEmbeddingLayer(trainable=train_elmo)(input_text)

  ## output layer
  output = tf.keras.layers.Dense(1, activation='sigmoid')(embedding)

  model = tf.keras.Model(inputs=input_text, outputs=output)

  sess = tf.compat.v1.keras.backend.get_session()
  init = tf.compat.v1.global_variables_initializer()
  sess.run(init)

  optimizer_fn = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001)
  model.compile(optimizer=optimizer_fn, loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [23]:
## Finetuning
finetune_elmo = create_model(train_elmo=True)
finetune_elmo.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 elmo_embedding_layer (Elmo  (None, 1024)              4         
 EmbeddingLayer)                                                 
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 1029 (4.02 KB)
Trainable params: 1029 (4.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
### creating a folder in current directory for saving model weights

folder_path ='./ELMo_model'

if os.path.exists(folder_path) == True:
    None
else:
    os.makedirs(folder_path)

In [ ]:
# ## saving weights

checkpoint_path = './ELMo_model/weight_{epoch:02d}.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                save_best_only=True,
                                                mode='max',
                                                monitor='accuracy')
early = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=5,restore_best_weights=True)

checkpt_list = [checkpoint, early]

In [ ]:
EPOCHS = 2
BATCH_SIZE = 10
STEPS_PER_EPOCH = len(train_labels)//BATCH_SIZE
VAL_STEPS = len(val_labels)//BATCH_SIZE

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(len(train_x)).batch(BATCH_SIZE)
val_dataset = train_dataset.shuffle(len(val_x)).batch(BATCH_SIZE)

In [ ]:
## 22-9-2023 retraining
finetune_elmo.fit(train_dataset, epochs=5, callbacks=checkpt_list)

Train on 301 steps
Epoch 1/2
301/301 [==============================] - 319s 1s/step - batch: 150.0000 - size: 1.0000 - loss: 0.6318 - accuracy: 0.6421
Epoch 2/2
301/301 [==============================] - 218s 724ms/step - batch: 150.0000 - size: 1.0000 - loss: 0.6278 - accuracy: 0.6514


In [24]:
finetune_elmo.load_weights('./ELMo_model/weight_05.hdf5')

In [25]:
# Get the input layer and encoder layer from the trained model
input_layer = finetune_elmo.input
embb_layer = finetune_elmo.layers[1].output

# Create a new model that takes the input layer and encoder layer as inputs and outputs the encoder layer's output
elmo_embb_model = tf.keras.Model(inputs=input_layer, outputs=embb_layer)

## FAISS

In [27]:
!pip install faiss-cpu  # for CPU version
# or
!pip install faiss-gpu  # for GPU version


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.3 MB/s eta 0:00:00


In [28]:
import faiss
import numpy as np


In [29]:
questions_embeddings = np.load('finetuned_elmo_question_embeddings.npy')

In [30]:
# Convert question embeddings to a numpy matrix
question_embeddings_matrix = np.vstack(questions_embeddings)

# Get the dimension of the embeddings
dimension = question_embeddings_matrix.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)

# Add the question embeddings to the index
index.add(question_embeddings_matrix)

In [31]:
question_embeddings_matrix.shape

(2000, 1024)

In [33]:
def find_top_similar_question_indices(user_input, model, index, k=5):
    # Encode the user input
    embedding = model.predict([user_input])

    # Search the index for the top k similar embeddings
    _, indices = index.search(embedding, k)

    # # Flatten the indices array
    indices = indices.flatten()

    return indices

## Groundtruth Ranking

In [35]:

ground_truth_df = pd.read_csv('Groundtruth_relevant_answers.csv')

In [36]:
# taking 100 random questions

twothousand_q = ground_truth_df.sample(2000, random_state=11)
twothousand_q

,Question,Relevant_Answer,question_id
2440,cumsum in numpy ndarray (edited),<p>The solution of @Ben.T juste above worked p...,62200330
7382,Should I always provide default value in php f...,<p>MySQL does not support having default value...,24361295
13064,How to use/install Command 'host' Docker Alpin...,<p>Alpine has a website called pkgs.alpinelinu...,66851224
18810,Lua String patterns - shorter code,<p>You can at least chain and line-wrap it:</p...,15137004
359,Trouble removing package from Visual Studio 2010,"<p>Here's a <a href=""http://docs.nuget.org/doc...",11960360
...,...,...,...
9714,How to display a text in android after a butto...,"<p>OK, fjust write onClick method in OnClickLi...",9123887
13991,Terraform - How do I reference my Output insid...,<p>The module name is <code>subnetwork</code> ...,70510621
3128,Need help parsing TypeScript syntax,"<p>It is a lot to unpack, here it is line by l...",62851642
11191,GameObject store its own location in a tile ba...,<p>I'd advise you to store the location in the...,16743466


In [38]:
#  scores of answer bodies

all_answersscore_df = pd.read_csv('/content/drive/Shareddrives/SEM 4/DATA 298B Code/All_answers_scores.csv')
all_answersscore_df

,question_id,question_title,answer_id,answer_body,score
0,13128,How can I combine several C/C++ libraries into...,27676078,"<p>On Linux, <a href=""https://en.wikipedia.org...",8
1,13128,How can I combine several C/C++ libraries into...,13136,<p>I'm not sure how to physically combine them...,0
2,13128,How can I combine several C/C++ libraries into...,22041,"<p>Maybe I'm misunderstanding, but don't you o...",0
3,13128,How can I combine several C/C++ libraries into...,13140,"<p>On Unix like systems, the ld and ar utiliti...",9
4,13128,How can I combine several C/C++ libraries into...,13206,<p>You could extract the object files from eac...,26
...,...,...,...,...,...
33208,73831805,Why are methods not valid constructors?,73832791,<p>As pointed out in the comments on the origi...,2
33209,73834279,No ability to publish newer build for external...,73834304,<p>I believe that there is possible to do more...,0
33210,73837100,How can I run multiple expressions inside of a...,73837248,<p>You're getting a syntax error due to the as...,-1
33211,73838452,Pad the missing data with NA for two matrices,73838548,<p>You can use <code>match</code> to find the ...,1


In [39]:
eval_q_list = twothousand_q['Question'].tolist()

In [40]:
def find_top_similar_answer_indices(user_input, model, index, k=5):
    # Encode the user input
    user_input_embedding = model.predict([user_input])

    # Search the index for the top k similar embeddings
    _, indices = index.search(user_input_embedding, k)

    # Flatten the indices array
    indices = indices.flatten()

    return indices

In [41]:
from sklearn.metrics import ndcg_score

In [47]:
## fucntion for eval

### creating function for eval

def ndcg_evaluation(question_list):

  counter = 1
  for user_question in question_list:
    print(counter)
    counter += 1
    ndcg_total = []


    ## getting the top 5 indices - questions
    top_question_indices = find_top_similar_question_indices([user_question], elmo_embb_model, index, k=5)

    #### Groundtruth ranks
    ## initiate score dataframe

    scores_df = pd.DataFrame({'score':[], 'answer_id':[], 'answer_body' : []})
    scores_df['answer_id'] = scores_df['answer_id'].astype(int)

    ## get scores
    for ind in top_question_indices:
      if len(scores_df) <= 5:
        q_id = ground_truth_df.loc[ind, 'question_id']
        filtered_df = all_answersscore_df[all_answersscore_df['question_id'] == q_id]
        # Select 'score','answr body' and 'answer_id' columns
        filtered_df = filtered_df[['score', 'answer_id','answer_body']]
        scores_df = pd.concat([scores_df,filtered_df], axis=0, ignore_index=True)
      else:
        break

    # Sort the scores DataFrame based on 'score' in descending order
    sorted_df = scores_df.sort_values(by='score', ascending=False)
    result_df = sorted_df.head(5)
    result_df.reset_index(drop=True, inplace=True)
    result_df = result_df.copy()
    result_df.loc[:, 'gt_relevance'] = list(range(len(result_df)-1, -1, -1))

    answer_ids = result_df['answer_id'].to_list()

    ## getting answer bodies
    answers_text_list = result_df['answer_body'].to_list()
    answers_embeddings = [elmo_embb_model.predict([[answer]]) for answer in answers_text_list]

    ## FAISS
    # Convert answer embeddings to a numpy matrix
    answer_embeddings_matrix = np.vstack(answers_embeddings)
    # Get the dimension of the embeddings
    dimension = answer_embeddings_matrix.shape[1]
    # Create a FAISS index
    answer_index = faiss.IndexFlatL2(dimension)
    # Add the question embeddings to the index
    answer_index.add(answer_embeddings_matrix)

    ## getting top answer indices
    top_answer_indices = find_top_similar_answer_indices([user_question], elmo_embb_model, answer_index, k=5)

    result_df['top_indices'] = top_answer_indices

    fetched_values = []
    for i in top_answer_indices:
      test_rel_score = result_df['gt_relevance'].iloc[i]
      fetched_values.append(test_rel_score)

    result_df['test_relevance'] = np.array(fetched_values)


    ndcg = ndcg_score([result_df['gt_relevance'].to_list()], [result_df['test_relevance'].to_list()])

    ndcg_total.append(ndcg)

  ndcg_mean = np.mean(ndcg_total)

  return round(ndcg_mean, 4)


ElMo model - FAISS Evaluation

In [49]:
overall_ndgc_score = []

In [50]:
## 0 - 500
elmo_ndcg_score1 = ndcg_evaluation(eval_q_list[:500])
overall_ndgc_score.append(elmo_ndcg_score1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [51]:
## 500 - 1000
elmo_ndcg_score2 = ndcg_evaluation(eval_q_list[500:1000])
overall_ndgc_score.append(elmo_ndcg_score2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [52]:
## 1000 - 1500
elmo_ndcg_score3 = ndcg_evaluation(eval_q_list[1000:1500])
overall_ndgc_score.append(elmo_ndcg_score3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [53]:
## 1500 - 2000
elmo_ndcg_score4 = ndcg_evaluation(eval_q_list[1500:2000])
overall_ndgc_score.append(elmo_ndcg_score4)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [54]:
overall_ndgc_score

[0.9751, 0.7104, 0.6199, 0.748]

In [55]:
print(f'The NDGC score of ELMo - FAISS model (for 2000 test questions) is {np.mean(overall_ndgc_score):.4f}')

The NDGC score of ELMo - FAISS model (for 2000 test questions) is 0.7633
